# Ingest results from bioinformatics run

`testuser1` performed perturb-seq and initiated a bioinformatics workflow run:

In [ ]:
!lamin login testuser1

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

## Upload raw fastq files

In [ ]:
bfx_run_output = ln.dev.datasets.dir_scrnaseq_cellranger(
    "perturbseq", basedir=ln.settings.storage, output_only=False
)
ln.track(ln.Transform(name="Chromium 10x upload", type="pipeline"))
ln.File(bfx_run_output.parent / "fastq/perturbseq_R1_001.fastq.gz").save()
ln.File(bfx_run_output.parent / "fastq/perturbseq_R2_001.fastq.gz").save()
ln.setup.login("testuser2")

## Bioinformatics run

In [ ]:
transform = ln.Transform(name="Cell Ranger", version="7.2.0", type="pipeline")
ln.track(transform)

In [ ]:
files = ln.File.filter(key__startswith="fastq/perturbseq").all()
filepaths = [file.stage() for file in files]

In [ ]:
output_files = ln.File.from_dir("./mycrispr/perturbseq/filtered_feature_bc_matrix/")
ln.save(output_files)

In [ ]:
output_files[0].view_lineage()

## Pre-process Cell Ranger outputs

In [ ]:
transform = ln.Transform(
    name="Preprocess Cell Ranger outputs", version="2.0", type="pipeline"
)
ln.track(transform)

In [ ]:
[f.stage() for f in output_files]
filepath = ln.dev.datasets.schmidt22_perturbseq(basedir=ln.settings.storage)

In [ ]:
file = ln.File(filepath, description="perturbseq counts")
file.save()

In [ ]:
efo = lb.ExperimentalFactor.lookup()

file.add_labels(efo.perturb_seq, "assay")
file.add_labels(efo.single_cell_rna_sequencing, "readout")

In [ ]:
is_experiment = ln.Label.filter(name="is_experiment").one()
is_biosample = ln.Label.filter(name="is_biosample").one()

exp2 = is_experiment.children.filter(description__contains="Perturb-seq")

biosample = is_biosample.children.get(name="S001")

file.add_labels(exp2, "experiment")
file.add_labels(biosample, "biosample")

In [ ]:
file.describe()

In [ ]:
file.view_lineage()